# Goal
-  Join the two tables on review id

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import graphlab as gl
%matplotlib inline

## Let's read data

### Listing details

In [157]:
listing_detail = pd.read_csv('data/listings_detail.csv', index_col=None)

In [159]:
print listing_detail.info()
print np.shape(listing_detail)
listing_detail.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7029 entries, 0 to 7028
Data columns (total 92 columns):
id                                  7029 non-null int64
listing_url                         7029 non-null object
scrape_id                           7029 non-null int64
last_scraped                        7029 non-null object
name                                7029 non-null object
summary                             6457 non-null object
space                               5652 non-null object
description                         7026 non-null object
experiences_offered                 7029 non-null object
neighborhood_overview               4371 non-null object
notes                               3212 non-null object
transit                             4571 non-null object
thumbnail_url                       6504 non-null object
medium_url                          6504 non-null object
picture_url                         7029 non-null object
xl_picture_url                      6504

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,1162609,https://www.airbnb.com/rooms/1162609,20151101181058,2015-11-02,Lovely One Bedroom Apartment,This is a big well designed space. Our neighbo...,This is a great one bedroom apartment. It is a...,This is a big well designed space. Our neighbo...,none,NaN,...,9,t,NaN,SAN FRANCISCO,f,strict,f,f,1,0.28
1,6032828,https://www.airbnb.com/rooms/6032828,20151101181058,2015-11-01,Historic Seacliff Home,Beautiful charmng multi- bedroom home in prest...,NaN,Beautiful charmng multi- bedroom home in prest...,none,NaN,...,NaN,t,NaN,SAN FRANCISCO,f,flexible,f,f,1,NaN
2,6938818,https://www.airbnb.com/rooms/6938818,20151101181058,2015-11-02,Best Secret in Town,Convenience is the key for your travel. Walkin...,The room is spacious and it is on the top leve...,Convenience is the key for your travel. Walkin...,none,It is in the city and close to everything. Par...,...,9,t,S. F. Short-Term Residential Rental Registrati...,SAN FRANCISCO,f,strict,f,f,1,2.33
3,8087607,https://www.airbnb.com/rooms/8087607,20151101181058,2015-11-02,Single Room Beautiful Beach Condo,"When I travel, my quiet beach side condo makes...",NaN,"When I travel, my quiet beach side condo makes...",none,NaN,...,NaN,t,NaN,SAN FRANCISCO,f,flexible,f,f,1,NaN
4,4781448,https://www.airbnb.com/rooms/4781448,20151101181058,2015-11-02,3 Bd 2.5 Ba Full Flat Condo w Views,Our family-friendly condo is located in San Fr...,"Open concept living, dining area has great cit...",Our family-friendly condo is located in San Fr...,none,We are located across the street from Lincoln ...,...,10,t,NaN,SAN FRANCISCO,t,strict,f,f,1,0.36


In [132]:
len(listing_detail.columns)

92

### Pick columns
- I hand-selected columns that would be useful for the later analyses. 

In [167]:
columns_listing_detail = pd.read_csv('data/columns.txt', names=['col']) #hand-selected columns from original data
columns_listing_detail = [item for item in columns_listing_detail['col']]

In [182]:
listings = listing_detail[columns_listing_detail]
listings.head()

,id,listing_url,name,summary,space,description,neighborhood_overview,notes,transit,host_id,...,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,reviews_per_month
0,1162609,https://www.airbnb.com/rooms/1162609,Lovely One Bedroom Apartment,This is a big well designed space. Our neighbo...,This is a great one bedroom apartment. It is a...,This is a big well designed space. Our neighbo...,NaN,NaN,NaN,6368122,...,89,9,9,10,10,9,9,f,strict,0.28
1,6032828,https://www.airbnb.com/rooms/6032828,Historic Seacliff Home,Beautiful charmng multi- bedroom home in prest...,NaN,Beautiful charmng multi- bedroom home in prest...,NaN,NaN,NaN,30384615,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,flexible,NaN
2,6938818,https://www.airbnb.com/rooms/6938818,Best Secret in Town,Convenience is the key for your travel. Walkin...,The room is spacious and it is on the top leve...,Convenience is the key for your travel. Walkin...,It is in the city and close to everything. Par...,The summer in San Francisco is cooler than any...,"If you are on budget, the public transportatio...",36381578,...,92,9,9,10,9,9,9,f,strict,2.33
3,8087607,https://www.airbnb.com/rooms/8087607,Single Room Beautiful Beach Condo,"When I travel, my quiet beach side condo makes...",NaN,"When I travel, my quiet beach side condo makes...",NaN,NaN,NaN,3264449,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,flexible,NaN
4,4781448,https://www.airbnb.com/rooms/4781448,3 Bd 2.5 Ba Full Flat Condo w Views,Our family-friendly condo is located in San Fr...,"Open concept living, dining area has great cit...",Our family-friendly condo is located in San Fr...,We are located across the street from Lincoln ...,2 night minimum stay please.,Easy bus access to the entire city with 3 bus ...,13112558,...,100,10,10,10,10,10,10,t,strict,0.36


### Review data

In [99]:
reviews = pd.read_csv('data/reviews.csv')
print reviews.info()
reviews.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143237 entries, 0 to 143236
Data columns (total 6 columns):
listing_id       143237 non-null int64
id               143237 non-null int64
date             143237 non-null object
reviewer_id      143237 non-null int64
reviewer_name    143237 non-null object
comments         143201 non-null object
dtypes: int64(3), object(3)
memory usage: 7.6+ MB
None


,listing_id,id,date,reviewer_id,reviewer_name,comments
0,1994427,10612780,2014-02-27,4918890,Katie,Great host. Provided special gift basket upon ...
1,1994427,49925683,2015-10-07,44613424,Steven,Great host. Met me at apartment. Place was cle...
2,1994427,50537634,2015-10-12,7616696,Marsilius,Syeda hosted my parents for four days during t...
3,1994427,51077613,2015-10-17,16703590,Robert,I had a great experience at Syeda's Airbnb! It...
4,1994427,52109034,2015-10-26,6183409,Steve,"Great location! Warm, welcoming host. This w..."


In [198]:
reviews = reviews.rename(columns={'listing_id': 'id', 
                        'id': 'review_id',})

In [199]:
print np.shape(listings)
print np.shape(reviews)

(7029, 53)
(143237, 6)


In [200]:
listings.head(1)

,id,listing_url,name,summary,space,description,neighborhood_overview,notes,transit,host_id,...,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,reviews_per_month
0,1162609,https://www.airbnb.com/rooms/1162609,Lovely One Bedroom Apartment,This is a big well designed space. Our neighbo...,This is a great one bedroom apartment. It is a...,This is a big well designed space. Our neighbo...,NaN,NaN,NaN,6368122,...,89,9,9,10,10,9,9,f,strict,0.28


In [201]:
reviews.head(1)

,id,review_id,date,reviewer_id,reviewer_name,comments
0,1994427,10612780,2014-02-27,4918890,Katie,Great host. Provided special gift basket upon ...


## Merge tables
- Inner join listings and reviews on "id"

In [214]:
df =  pd.DataFrame.merge(reviews, listings, left_on='id', right_on='id', how='inner')
print np.shape(df)
df.head()

(143237, 58)


,id,review_id,date,reviewer_id,reviewer_name,comments,listing_url,name,summary,space,...,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,reviews_per_month
0,1994427,10612780,2014-02-27,4918890,Katie,Great host. Provided special gift basket upon ...,https://www.airbnb.com/rooms/1994427,One Bedroom apartment,Charming one bedroom in the Mission! Close to...,The bedroom fits two on the full sized bed and...,...,92,9,9,10,10,10,9,f,strict,0.24
1,1994427,49925683,2015-10-07,44613424,Steven,Great host. Met me at apartment. Place was cle...,https://www.airbnb.com/rooms/1994427,One Bedroom apartment,Charming one bedroom in the Mission! Close to...,The bedroom fits two on the full sized bed and...,...,92,9,9,10,10,10,9,f,strict,0.24
2,1994427,50537634,2015-10-12,7616696,Marsilius,Syeda hosted my parents for four days during t...,https://www.airbnb.com/rooms/1994427,One Bedroom apartment,Charming one bedroom in the Mission! Close to...,The bedroom fits two on the full sized bed and...,...,92,9,9,10,10,10,9,f,strict,0.24
3,1994427,51077613,2015-10-17,16703590,Robert,I had a great experience at Syeda's Airbnb! It...,https://www.airbnb.com/rooms/1994427,One Bedroom apartment,Charming one bedroom in the Mission! Close to...,The bedroom fits two on the full sized bed and...,...,92,9,9,10,10,10,9,f,strict,0.24
4,1994427,52109034,2015-10-26,6183409,Steve,"Great location! Warm, welcoming host. This w...",https://www.airbnb.com/rooms/1994427,One Bedroom apartment,Charming one bedroom in the Mission! Close to...,The bedroom fits two on the full sized bed and...,...,92,9,9,10,10,10,9,f,strict,0.24


In [210]:
print np.shape(df)
print np.shape(df2)

(143237, 58)
(144568, 58)


In [216]:
pwd

u'/Users/kimitakanakazawa/Dropbox/galvanizeU/FP6003_6004'

## Export with csv format

In [218]:
# listings.to_csv('data/listings_new.csv')
# reviews.to_csv('data/reviews_new.csv')
# df.to_csv('data/df_new.csv')